## Code : Expert Agent
The following code can be find in the `expert_farm1.py` in `examples/` folder.

### Imports:
- `AgentWithSimplePolicy, AgentManager, evaluate_agents` : for rlberry use
- `Farm1` : the model environment
- `display_evaluation_result` : print the mean, median and std of the evaluation

In [30]:
from rlberry.agents import AgentWithSimplePolicy
from rlberry.manager import AgentManager, evaluate_agents
from rlberry_farms import Farm1
import numpy as np
from rlberry_farms.utils import display_evaluation_result

### Settings :
We'll use the 'Farm1' environment <br/>
For this specific expert agent, we can decide the day we will start the policy in 'starting_day_for_policy'. 


In [31]:
env_ctor, env_kwargs = Farm1, {}
starting_day_for_policy = 100

### Class Expert Agent :

To be compatible with our system, the expert Agent should be a RLBerry agent. So it have to extend `AgentWithSimplePolicy`, and implement `_init_, _fit_`, and `_policy_`.
- init : we can use it to manage specific settings (here 'starting_day_for_policy').
- fit : we use _classic_ rlberry _**fit**_, but it will be called with a budget of 1 (because expert agent don't need training).
- policy : it's where you define how your agent choose an action from the given observation.

---

**HERE, the policy of this agent is to : <br />**
- 1) wait until the 'starting_day_for_policy'. <br />
- 2) Watering, herbicide, pesticide, fertilize,sow, watering. <br />
- 3) Waiting the plant got fruits, wait 4 more days, then harvest. <br />
        
**HERE, the available actions are :**
- 0) Do nothing <br />
- 1) Pour 1L of water <br />
- 2) Pour 5L of water <br />
- 3) Harvest the plant <br />
- 4) sow <br />
- 5) Fertilizer <br />
- 6) Herbicide <br />
- 7) Pesticide <br />
- 8) Remove weeds by hand <br />

**HERE, the content of the observation array is : <br />**
- observation[0] : Day (from 1 to 365)<br />
- observation[1] : Mean air temperature (°C)<br /> 
- observation[2] : Min air temperature (°C)<br />
- observation[3] : Max air temperature (°C)<br />
- observation[4] : Rain amount<br />
- observation[5] : Sun-exposure (from 1 to 5)<br />
- observation[6] : Consecutive dry day (int)<br />
- observation[7] : Stage of growth of the plant<br />
- observation[8] : Number of fruits (int)<br />
- observation[9] : Size of the plant in cm<br />
- observation[10] : Soil wet_surface (m2.day-1)<br />
- observation[11] : fertilizer amount (kg)<br />
- observation[12] : Pollinators occurrence (bin)<br />
- observation[13] : Weeds grow (nb)<br />
- observation[14] : Weeds flowers (nb)<br />
- observation[15] : weight of fruits<br />
- observation[16] : microlife health index (%)<br />

In [35]:

class ExpertAgent(AgentWithSimplePolicy):
    name = "ExpertAgentFarm1"
    fruit_stage_duration_count = 0

    def __init__(self, env, starting_day_for_policy=0, **kwargs):
        AgentWithSimplePolicy.__init__(self, env ,**kwargs)
        self.starting_day_for_policy = starting_day_for_policy

    def fit(self, budget=100, **kwargs):
        observation = self.env.reset()
        episode_reward = 0
        for ep in range(int(budget)):
            action = self.policy(observation)
            observation, reward, done, info = self.env.step(action)
            episode_reward += reward
            if done:
                self.writer.add_scalar("episode_rewards", episode_reward, ep)
                episode_reward = 0
                self.env.reset()


    def policy(self, observation):
        next_action = 0 #default
        if observation[0] == starting_day_for_policy:
            next_action = 2  # 5L of water
        elif observation[0] == starting_day_for_policy+1:
            next_action = 6  # herbicide
        elif observation[0] == starting_day_for_policy+2:
            next_action = 7  # pesticide
        elif observation[0] == starting_day_for_policy+3:
            next_action = 5  # Fertilizer
        elif observation[0] == starting_day_for_policy+4:
            next_action = 4  # sow
        elif observation[0] == starting_day_for_policy+5:
            next_action = 1  # 1L of water
        elif observation[0] > starting_day_for_policy+5:
            if observation[7] in [6, 7, 8, 9]:
                if (self.fruit_stage_duration_count > 4):
                    next_action = 3  # harvesting
                    self.fruit_stage_duration_count = 0
                else:
                    self.fruit_stage_duration_count += 1
            
        return next_action

### Class Agent :
Create an Agent (called 'Agent') that heritate from ExpertAgent, to match the name and the expected signature for the challenge. 

In [36]:
class Agent(ExpertAgent):
    def __init__(self,env,**kwargs):
        ExpertAgent.__init__(self, env, starting_day_for_policy,**kwargs)

### Main code:
Run your agent through the [RLBerry agent manager](https://rlberry.readthedocs.io/en/latest/generated/rlberry.manager.AgentManager.html#rlberry.manager.AgentManager) with the setting you need.
_(here we use `n_fit=1` : because expert agent don't need training)_

then display the results of the evaluation.

In [37]:
manager = AgentManager(
    Agent,
    (env_ctor, env_kwargs),
    agent_name="ExpertAgentFarm1",
    fit_budget=1,
    eval_kwargs=dict(eval_horizon=365),
    n_fit=1,
    output_dir="expert_farm1_results",
)
manager.fit()
evaluation = evaluate_agents([manager], n_simulations=128, plot=False).values
#np.savetxt("expert_farm1.out", np.array(evaluation), delimiter=",")
display_evaluation_result(evaluation)


[INFO] 10:11: Running AgentManager fit() for ExpertAgentFarm1 with n_fit = 1 and max_workers = None. 
/home/jteigny/anaconda3/envs/rlberry_farms/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/jteigny/anaconda3/envs/rlberry_farms/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(
/home/jteigny/anaconda3/envs/rlberry_farms/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/jteigny/anaconda3/envs/rlberry_farms/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(
[INFO] 10:11: ... trained! 
/home/jteigny/anaconda3/envs/rlberry_farms/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32


evaluation_mean : 20.868188622446212
evaluation_median : 20.518331395108568
evaluation_std : 6.883607528055441


<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>